In [1]:
import os.path
import numpy as np
import pandas as pd

from pathlib import Path
from edisgo.io.simBEV_import import get_ags

C:\Users\Kilian.Helfenbein\.conda\envs\eDisGo_env\lib\site-packages\sqlalchemy\ext\declarative\base.py:436: SAWarning: Ignoring declarative-like tuple value of attribute 'segment': possibly a copy-and-paste error with a comma accidentally placed at the end of the line?
  "accidentally placed at the end of the line?" % k
C:\Users\Kilian.Helfenbein\.conda\envs\eDisGo_env\lib\site-packages\sqlalchemy\ext\declarative\base.py:436: SAWarning: Ignoring declarative-like tuple value of attribute 'cable': possibly a copy-and-paste error with a comma accidentally placed at the end of the line?
  "accidentally placed at the end of the line?" % k


In [2]:
data_dir = Path(r"\\192.168.10.221\Daten_flexibel_02\simbev_results")

sub_dir = "standing_times"

scenarios = [
    "Electrification_2050_simbev_run",
    "Electrification_2050_sensitivity_low_work_simbev_run",
    "Mobility_Transition_2050_simbev_run",
    "Szenarette_Kleinwagen_2050_simbev_run",
    "Reference_2050_simbev_run",
    "NEP_C_2035_simbev_run",
]

data_dirs = [
    Path(os.path.join(data_dir, scenario, sub_dir)) for scenario in scenarios
]

In [3]:
scenario_data_path = Path(r"C:\Users\Kilian.Helfenbein\git\gogs\245\TR\scenario_data\RampUp_scenario_Electrification_2050.csv")
df_scenario_data = pd.read_csv(scenario_data_path, index_col=[0]).set_index("AGS")

In [4]:
df_standing_time = pd.DataFrame(columns=["location", "charge_time", "charging_time"])

for data_dir in data_dirs:
    _, ags_dirs = get_ags(data_dir)
    
    count_cars = 0
    
    for count, ags_dir in enumerate(ags_dirs):
        ags = int(ags_dir.parts[-1])
        
        regio_type = df_scenario_data.at[ags, "RegioStaR7"]
        
        car_csvs = os.listdir(ags_dir)
                
        csvs = [Path(os.path.join(ags_dir, csv)) for csv in car_csvs]
        
        for csv in csvs:
            df = pd.read_csv(csv, index_col=[0])
            
            df = df[df.chargingdemand > 0]
            
            if not df.empty:
                df["charging_time"] = df.chargingdemand.divide(
                    df.netto_charging_capacity.divide(4)
                ).round(1).apply(np.ceil).astype(int)

                df_standing_time = df_standing_time.append(
                    df[["location", "charge_time", "charging_time"]],
                    ignore_index=True,
                )
                                    
        print("AGS {} in scenario {} is done.".format(ags_dir.parts[-1], ags_dir.parts[-3][:-11]))
    
    break

df_standing_time.rename(
    columns={
        "charge_time": "standing_time"
    },
    inplace=True,
)
df_standing_time = df_standing_time.sort_values(by="location").set_index("location")
df_standing_time.to_csv("simbev_analysis_standing_time_all.csv")

AGS 03351002 in scenario Electrification_2050 is done.
AGS 03351003 in scenario Electrification_2050 is done.
AGS 03351008 in scenario Electrification_2050 is done.
AGS 03351015 in scenario Electrification_2050 is done.
AGS 03351016 in scenario Electrification_2050 is done.
AGS 03351017 in scenario Electrification_2050 is done.
AGS 08115029 in scenario Electrification_2050 is done.
AGS 08115041 in scenario Electrification_2050 is done.
AGS 08115054 in scenario Electrification_2050 is done.
AGS 08325011 in scenario Electrification_2050 is done.
AGS 08325049 in scenario Electrification_2050 is done.
AGS 08325060 in scenario Electrification_2050 is done.
AGS 08325069 in scenario Electrification_2050 is done.
AGS 09184119 in scenario Electrification_2050 is done.
AGS 09471133 in scenario Electrification_2050 is done.
AGS 09674130 in scenario Electrification_2050 is done.
AGS 09674184 in scenario Electrification_2050 is done.
AGS 09674190 in scenario Electrification_2050 is done.
AGS 096742

In [5]:
df_loc_standing_time = pd.DataFrame(
    columns=["location", "mean_standing_time", "std_standing_time", "mean_charging_time", "std_charging_time"]
)

for idx in df_standing_time.index.unique():
    df_loc_standing_time.loc[len(df_loc_standing_time)] = [
        idx,
        df_standing_time.loc[idx].standing_time.mean(),
        df_standing_time.loc[idx].standing_time.std(),
        df_standing_time.loc[idx].charging_time.mean(),
        df_standing_time.loc[idx].charging_time.std(),
    ]
df_loc_standing_time = df_loc_standing_time.set_index("location")
df_loc_standing_time.to_csv("simbev_analysis_standing_time.csv")

In [6]:
df_loc_standing_time

,mean_standing_time,std_standing_time,mean_charging_time,std_charging_time
location,,,,
0_work,29.314730,14.785274,2.235028,1.897567
1_business,17.480420,23.363520,4.259790,2.934488
2_school,24.852732,15.636619,5.615202,4.269367
3_shopping,8.504902,15.036961,3.207879,1.910328
4_private/ridesharing,16.693077,22.386447,3.549032,2.361801
5_leisure,24.061404,24.616873,4.250977,2.987726
6_home,35.791256,22.892813,2.830512,2.281630
7_charging_hub,1.000000,0.000000,1.000000,0.000000


In [4]:
bev_mini_con = 0.1185
bev_medium_con = 0.1482
bev_luxury_con = 0.1778

lr_klein_con_lst = []
lr_mitte_con_lst = []
sr_mitte_con_lst = []

count_cars_lst = []

for data_dir in data_dirs:
    _, ags_dirs = get_ags(data_dir)
    
    count_cars = 0
    
    for count, ags_dir in enumerate(ags_dirs):
        ags = int(ags_dir.parts[-1])
        
        regio_type = df_scenario_data.at[ags, "RegioStaR7"]
        
        car_csvs = os.listdir(ags_dir)
                
        bev_csvs = [csv for csv in car_csvs if "BEV" in csv]
        
        bev_mini_csvs = [Path(os.path.join(ags_dir, csv)) for csv in bev_csvs if "mini" in csv]
        
        bev_medium_csvs = [Path(os.path.join(ags_dir, csv)) for csv in bev_csvs if "medium" in csv]
        
        bev_luxury_csvs = [Path(os.path.join(ags_dir, csv)) for csv in bev_csvs if "luxury" in csv]
        
        for car in bev_mini_csvs:
            df = pd.read_csv(car)
            
            if regio_type == "LR_Klein":
                lr_klein_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_mini_con)
            elif regio_type == "LR_Mitte":
                lr_mitte_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_mini_con)
            else:
                sr_mitte_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_mini_con)
                    
        for car in bev_medium_csvs:
            df = pd.read_csv(car)
            
            if regio_type == "LR_Klein":
                lr_klein_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_medium_con)
            elif regio_type == "LR_Mitte":
                lr_mitte_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_medium_con)
            else:
                sr_mitte_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_medium_con)
        
        for car in bev_luxury_csvs:
            df = pd.read_csv(car)
            
            if regio_type == "LR_Klein":
                lr_klein_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_luxury_con)
            elif regio_type == "LR_Mitte":
                lr_mitte_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_luxury_con)
            else:
                sr_mitte_con_lst.append(df.chargingdemand.sum() * 365 / 7 / bev_luxury_con)
            
        del df
                        
        print("AGS {} in scenario {} is done.".format(ags_dir.parts[-1], ags_dir.parts[-3][:-11]))


AGS 03351002 in scenario Electrification_2050 is done.
AGS 03351003 in scenario Electrification_2050 is done.
AGS 03351008 in scenario Electrification_2050 is done.
AGS 03351015 in scenario Electrification_2050 is done.
AGS 03351016 in scenario Electrification_2050 is done.
AGS 03351017 in scenario Electrification_2050 is done.
AGS 08115029 in scenario Electrification_2050 is done.
AGS 08115041 in scenario Electrification_2050 is done.
AGS 08115054 in scenario Electrification_2050 is done.
AGS 08325011 in scenario Electrification_2050 is done.
AGS 08325049 in scenario Electrification_2050 is done.
AGS 08325060 in scenario Electrification_2050 is done.
AGS 08325069 in scenario Electrification_2050 is done.
AGS 09184119 in scenario Electrification_2050 is done.
AGS 09471133 in scenario Electrification_2050 is done.
AGS 09674130 in scenario Electrification_2050 is done.
AGS 09674184 in scenario Electrification_2050 is done.
AGS 09674190 in scenario Electrification_2050 is done.
AGS 096742

AGS 13076089 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076096 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076109 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076120 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076135 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076151 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076164 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076165 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076166 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 03351002 in scenario Reference_2050 is done.
AGS 03351003 in scenario Reference_2050 is done.
AGS 03351008 in scenario Reference_2050 is done.
AGS 03351015 in scenario Reference_2050 is done.
AGS 03351016 in scenario Reference_2050 is done.
AGS 03351017 in scenario Reference_2050 is done.
AGS 08115029 in scenario Reference_2050 is done.
AGS 08115041 in scenario Reference_2050 is done.
AGS 08115054 in scenario Reference_2050 is done.
AGS 083250

In [5]:
df = pd.DataFrame(
    {
        "lr_klein": pd.Series(lr_klein_con_lst),
        "lr_mitte": pd.Series(lr_mitte_con_lst),
        "sr_mitte": pd.Series(sr_mitte_con_lst),
    }
)

df.to_csv("simbev_analysis_distance.csv")

In [6]:
df.mean()

lr_klein    12352.560339
lr_mitte    11500.474989
sr_mitte    11660.256231
dtype: float64

In [7]:
df

,lr_klein,lr_mitte,sr_mitte
0,12658.064154,8869.608672,15102.395846
1,8800.792392,31322.453527,8834.224255
2,10506.670815,26810.861039,10112.175825
3,10798.428945,9757.094064,7825.690277
4,4356.238698,10867.459890,14165.051053
5,7144.841448,13941.157260,0.000000
6,25250.520128,7097.914068,7276.510892
7,7286.267007,10444.971223,10526.666257
8,16571.662503,8021.324162,11024.716211
9,0.000000,8906.694768,7975.391020
